In [3]:
import requests
from time import sleep

def get_token():
    headers = {
        'authority': 'clerk.suno.ai',
        'accept': '*/*',
        'accept-language': 'en-BR,en;q=0.9,pt-BR;q=0.8,pt;q=0.7,es-ES;q=0.6,es;q=0.5,de-LI;q=0.4,de;q=0.3,en-GB;q=0.2,en-US;q=0.1,sk;q=0.1',
        'content-type': 'application/x-www-form-urlencoded',
        #'cookie': 'Adquire this from your network tab for Suno',
        'origin': 'https://app.suno.ai',
        'referer': 'https://app.suno.ai/',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    }
    #Adquire this from your logged-in session on Suno
    response = requests.post('https://clerk.suno.ai/v1/client/sessions...', headers=headers)
    print(response)
    token = response.json()["jwt"]
    return token

In [4]:
def get_model_id(general_prompt="", lyrics="", style="", instrumental=False):
    token = get_token()
    url = "https://studio-api.suno.ai/api/generate/v2/"

    headers = {
        "authority": "studio-api.suno.ai",
        "accept": "*/*",
        "accept-language": "en-BR,en;q=0.9,pt-BR;q=0.8,pt;q=0.7,es-ES;q=0.6,es;q=0.5,de-LI;q=0.4,de;q=0.3,en-GB;q=0.2,en-US;q=0.1,sk;q=0.1",
        "authorization": f"Bearer {token}",
        "content-type": "text/plain;charset=UTF-8",
        "origin": "https://app.suno.ai",
        "referer": "https://app.suno.ai/",
        "sec-ch-ua": '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"macOS"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    }
    if(lyrics):
        data = f'{{"mv":"chirp-v3-alpha","prompt":"{lyrics}","tags":"{style}","make_instrumental":{str(instrumental).lower()}}}'
    else:
        data = f'{{"gpt_description_prompt":"{general_prompt}","mv":"chirp-v3-alpha","prompt":"","make_instrumental":{str(instrumental).lower()}}}'
    
    print(data)
    response = requests.post(url, headers=headers, data=data)
    task_response = response.json()
    clip_ids = []
    for clip in task_response["clips"]:
        clip_ids.append(clip["id"])
    return clip_ids

In [5]:
def get_feed(clip_ids):

    def match_results(clip_ids, feed_response):
        result_data = []
        for clip_id in clip_ids:
            for model_in_feed in feed_response:
                if(model_in_feed["id"] == clip_id):
                    if(model_in_feed["audio_url"]):
                        result_data.append(model_in_feed)
        return result_data if len(result_data) == 2 else False

    token = get_token()
    headers = {
        'authority': 'studio-api.suno.ai',
        'accept': '*/*',
        'accept-language': 'en-BR,en;q=0.9,pt-BR;q=0.8,pt;q=0.7,es-ES;q=0.6,es;q=0.5,de-LI;q=0.4,de;q=0.3,en-GB;q=0.2,en-US;q=0.1,sk;q=0.1',
        'authorization': f'Bearer {token}',
        'origin': 'https://app.suno.ai',
        'referer': 'https://app.suno.ai/',
        'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    }

    response = requests.get('https://studio-api.suno.ai/api/feed/', headers=headers)
    
    feed_response = response.json()
    return match_results(clip_ids, feed_response)

In [6]:

def get_suno_songs(prompt="", instrumental=False, lyrics="", style=""):
    clip_ids = get_model_id(prompt, lyrics, style, instrumental)
    counter = 0

    while True:
        matched_results = get_feed(clip_ids)
        if(matched_results):
            break
        else:
            counter+=1
            if(counter > 250):
                print("Aborted after 250s")
                break
            sleep(1)

    audio_urls = []
    for result in matched_results:
        audio_urls.append(result["audio_url"])
    return audio_urls

In [90]:
lyrics = '''fuck artificial intelligence'''
get_suno_songs(style="8-bit chipmunk dogs", )

{"mv":"chirp-v3-alpha","prompt":"fuck artificial intelligence","tags":"dope hip-hop","make_instrumental":false}


['https://audiopipe.suno.ai/?item_id=6a60adde-776f-457b-b74b-30fde2eb5041',
 'https://audiopipe.suno.ai/?item_id=6f5adb3a-3df3-41cb-9da3-de2f1177482b']